In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')
    
from datetime import datetime
import pytz
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from collections import defaultdict
from datetime import timedelta
    
import map_utils
    
from gen_lib import data_saver, generation_utils
from gen_lib.config import *
from gen_lib.locations import LocationData
from gen_lib.grid_generator import generate_location_heatmap, VariationConfig, GridGeneratorEnhanced
from gen_lib.trends import TrendConfig, TrendGenerator, save_trend_data
from gen_lib.time_series import TimeSeriesHeatmapGenerator, TimeSeriesConfig

from trend_detection.preprocessing import preprocess_text
from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN

/Users/viktor/workspace/ds2/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [12]:
ids = list(map(int, open('../trend_detection/debug/1733017526822-2_3_4_7').read().split(',')))
len(ids)

289

In [14]:
df = pd.read_json('../trend_detection/data/trend_messages_v24.json')
df = df[df['id'].isin(ids)]
df.shape

(289, 8)

In [18]:
processed_messages = []

for m in df["text"]:
    processed_messages.append(preprocess_text(m))

df["text_processed"] = processed_messages

In [23]:
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = []

for text in df['text_processed']:
    embeddings.append(model.encode([text])[0])

df['embeddings'] = embeddings

In [25]:
df.head()

,d_trend_id,d_location_id,topic,timestamp,lon,lat,text,id,text_processed,embeddings
3624,2,8,SPORT,2010-01-01 05:00:20+00:00,-87.677280,41.689515,"""Just witnessed the craziest soccer game ever!...",3625,witnessed craziest soccer game ever var saved ...,"[0.021767572, 0.044011142, -0.009903417, -0.05..."
3625,2,8,SPORT,2010-01-01 05:00:30+00:00,-87.624306,41.895654,"""Still can't believe what I just saw! World Cu...",3626,still cant believe saw world cup drama finest ...,"[0.0030337672, -0.015120118, -0.035581466, -0...."
3626,2,8,SPORT,2010-01-01 05:01:54+00:00,-87.725395,41.738493,"""Just witnessed history in the making! Can't b...",3627,witnessed history making cant believe drama co...,"[0.02497915, 0.04186446, -0.026178243, -0.0108..."
3627,2,8,SPORT,2010-01-01 05:03:10+00:00,-87.709274,41.918114,"""Mind-blowing finish to the #WorldCupFinal! Ca...",3628,mindblowing finish worldcupfinal cant believe ...,"[0.0037705002, 0.023138858, -0.004089883, -0.0..."
3628,2,8,SPORT,2010-01-01 05:03:42+00:00,-87.659007,41.908836,"""Oh wow, what a heart-stopping #WorldCupFinal!...",3629,wow heartstopping worldcupfinal cant believe p...,"[-0.0005467424, 0.059333127, 0.0026965863, -0...."


In [35]:
messages = df.to_dict(orient='records')

In [27]:
clustering = DBSCAN(eps=0.7, min_samples=10)
labels = clustering.fit_predict(embeddings)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [75]:
unique_labels = set(labels)
for label in unique_labels:
    if label == -1:  # noise
        continue

    cluster_messages = [m for i, m in enumerate(messages) if labels[i] == label]
    print(label, set([m['d_trend_id'] for m in cluster_messages]), len(cluster_messages))

0 {3} 84
1 {4} 66
2 {7} 63


In [66]:
for el in messages:
    # del el['timestamp']
    # del el['lon']
    # del el['lat']
    el['embeddings'] = list(map(float, el['embeddings']))

In [73]:
with open("../trend_detection/debug/debug.jsonl", "a") as f:
    for el in messages:
        f.write(json.dumps(el) + "\n")

In [41]:
messages[:2]

[{'d_trend_id': 2,
  'd_location_id': 8,
  'topic': 'SPORT',
  'timestamp': Timestamp('2010-01-01 05:00:20+0000', tz='UTC'),
  'lon': -87.6772795027,
  'lat': 41.689515031,
  'text': '"Just witnessed the craziest soccer game ever! VAR saved the day with drama in overtime, cementing a World Cup champion. Can\'t believe what I just saw. #WorldCupFinal #Sports"',
  'id': 3625,
  'text_processed': 'witnessed craziest soccer game ever var saved day drama overtime cementing world cup champion cant believe saw worldcupfinal sports',
  'embeddings': array([ 2.17675716e-02,  4.40111421e-02, -9.90341697e-03, -5.53398281e-02,
          1.17308848e-01,  6.35771602e-02,  1.01754889e-01,  6.09112568e-02,
          1.01646245e-01,  4.97175045e-02, -9.10506546e-02, -1.86244939e-02,
          1.44162597e-02,  1.00963309e-01, -1.81704145e-02, -8.49486515e-02,
         -1.07493527e-01, -5.00376560e-02, -1.13939703e-01, -7.40194023e-02,
          5.30457348e-02, -4.88095395e-02, -3.90845053e-02, -6.951125